#### Explicando Algoritmo Apriori (Associação) com base no curso de ML do Jones Granatyr

In [1]:
import pandas as pd
d = {'Leite': [False, True, False, True, False, False, False, False, False, False],
     'Café': [True, False, True, True, False, False, False, False, False, False],
     'Cerveja': [False, True, False, False, True, False, False, False, False, False],
     'Pão': [True, True, True, True, False, False, True, False, False, False],
     'Manteiga': [True, True, True, True, False, True, False, False, False, False],
     'Arroz': [False, False, False, False, False, False, False, False, True, True],
     'Feijão': [False, False, False, False, False, False, False, True, True, False]}
df = pd.DataFrame(data=d)
del d
df

,Leite,Café,Cerveja,Pão,Manteiga,Arroz,Feijão
0,False,True,False,True,True,False,False
1,True,False,True,True,True,False,False
2,False,True,False,True,True,False,False
3,True,True,False,True,True,False,False
4,False,False,True,False,False,False,False
5,False,False,False,False,True,False,False
6,False,False,False,True,False,False,False
7,False,False,False,False,False,False,True
8,False,False,False,False,False,True,True
9,False,False,False,False,False,True,False


#### Apriori - Fase 1 (Suporte)
##### O suporte é a medida que indica a frequência de um conjunto de itens<br> Iremos considerar apenas os registros que mais aparecem na base de dados e desconsiderar aqueles que menos aparecem.<br> e o suporte o usuário que especifica. Neste exemplo é considerado apenas os registros >0.5 (50% de freq.)

##### Passo 1: Calcular o suporte de conjunto com apenas 1 Itemset

In [2]:
# Criando um novo DataFrame apenas de Quantidade de Frequencia
# Qnt
df_sup = pd.DataFrame(
                      index=[df.columns], 
                      columns=['Qnt', 'Suporte'])
for item in df_sup.index:
    item = item[0]
    df_sup['Qnt'][item] = df.loc[df[item]].count()[0]
# Freq
df_sup.Suporte = df_sup.Qnt / len(df)

#### Tabela de Suporte

In [3]:
df_sup

,Qnt,Suporte
Leite,2,0.2
Café,3,0.3
Cerveja,2,0.2
Pão,5,0.5
Manteiga,5,0.5
Arroz,2,0.2
Feijão,2,0.2


##### Neste caso, vamos apenas considerar items com Suporte >= 0.3, ou seja, iremos desconsiderar o Leite, Cerveja, Arroz e Feijão.

In [4]:
del df['Leite'], df['Cerveja'], df['Arroz'], df['Feijão']

In [5]:
df

,Café,Pão,Manteiga
0,True,True,True
1,False,True,True
2,True,True,True
3,True,True,True
4,False,False,False
5,False,False,True
6,False,True,False
7,False,False,False
8,False,False,False
9,False,False,False


#### Passo 2 da Fase 1 (Suporte): Calcular o suporte de conjuntos com 2 items.

In [15]:
# Obtendo o suporte do conjunto Café e Pão:
cafepao = len(df.loc[(df['Café']) & (df['Pão'])]) / len(df)
print('Café, pão: ',cafepao)
# Obtendo o suporte do conjunto Café e Manteiga:
cafemanteiga = len(df.loc[(df['Café']) & (df['Manteiga'])]) / len(df)
print('Café, manteiga: ',cafemanteiga)
# Obtendo o suporte do conjunto Manteiga e Pão:
paomanteiga = len(df.loc[(df['Manteiga']) & (df['Pão'])]) / len(df)
print('Manteiga, pão: ',paomanteiga)

Café, pão:  0.3
Café, manteiga:  0.3
Manteiga, pão:  0.4


#### Passo 3 da Fase 1 (Suporte): Calcular o suporte de conjuntos com 3 items.

In [16]:
# Obtendo o suporte do conjunto Café, Pão e Manteiga:
cafepaomanteiga = len(df.loc[(df['Café']) & (df['Pão']) & (df['Manteiga'])]) / len(df)
print('Café, pão, manteiga: ',cafepaomanteiga)

Café, pão, manteiga:  0.3


#### Apriori - Fase 2 (Confiança)
##### A confiança é a medida que define a probabilidade de ocorrência de combinação de um item, sua fórmula:</br>  <b>confiança = Números com X e Y / Número total de registros com X</b>
    
##### é estipulado a confiança >= 0.8 (80%)
##### Definindo as regras (2 Items): <br> SE CAFÉ ENTÃO PÃO - 3 / 3 = 1,0 (REGRA 100%)<br> SE PÃO ENTÃO CAFÉ - 3 / 5 = 0,6 (REGRA 60%)<br>SE CAFÉ ENTÃO MANTEIGA - 3 / 3 = 1,0<br> SE MANTEIGA ENTÃO PÃO - 3 / 5 = 0,6<br>SE PÃO ENTÃO MANTEIGA - 4 / 5 = 0,8<br>SE MANTEIGA ENTÃO PÃO - 4 / 5 = 0,8
##### Definindo as regras (3 Items): <br> SE CAFÉ, PÃO ENTÃO MANTEIGA - 3 / 3 = 1,0 <br> SE CAFÉ, MANTEIGA ENTÃO PÃO - 3 / 3 = 1,0 <br> SE PÃO, MANTEIGA ENTÃO CAFÉ - 3 / 4 = 0,75 <br> SE CAFÉ ENTÃO PÃO, MANTEIGA = 3 / 5 = 0,6 <br> SE MANTEIGA ENTÃO CAFÉ, PÃO - 3 / 5 = 0,6

#### Apriori - Lift (Vindo do verbo Elevar)
##### É utilizado para medir o quão confiável uma regra de associação é. Sua fórmula: <br> Lift A > B (o quão mais frequente torna-se B quando A ocorre) <br> Lift = confiança(A > B)/suporte(B) <br> Verificando o exemplo: <br> SE café ENTÃO pão:
##### confiança = 3/3 = 1,0 -> lift(1/0,5) = 2 (clientes que compram café tem 2x mais chances de comprarem pão)